<h1>A Recurrent Neural Network example using tensorflow</h1>
<p>This example also uses MNIST data to create the RNN that will recognize hand-written digits.
</p>
<p>The image data looks like this:</p>

<img src="../img/MNIST.png">

## RNN Overview

<img src="../img/RNN.png"/>
<p>A recurrent neural network (RNN) is a class of artificial neural network where connections between nodes form a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behavior. Unlike feedforward neural networks, RNNs can use their internal state (memory) to process sequences of inputs. This makes them applicable to tasks such as unsegmented, connected handwriting recognition or speech recognition.</p>

In [1]:
#  First import tensorflow and the rnn package
import tensorflow as tf
from tensorflow.contrib import rnn

# Then import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Let's establish our training parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# And our network parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [3]:
# Initialize weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
# This function is our basic RNN 

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [5]:
# Establish some needed variables
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [6]:
# Let's find our start time so we can see how long the training takes
import time
start = time.time()

# And now we start training

with tf.Session() as session:

    # Run the initializer
    session.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        session.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = session.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        session.run(accuracy, feed_dict={X: test_data, Y: test_label}))
    
    print("Training took", time.time()-start, "seconds.")

Step 1, Minibatch Loss= 2.9015, Training Accuracy= 0.070
Step 200, Minibatch Loss= 2.1878, Training Accuracy= 0.266
Step 400, Minibatch Loss= 1.9873, Training Accuracy= 0.391
Step 600, Minibatch Loss= 1.8796, Training Accuracy= 0.430
Step 800, Minibatch Loss= 1.7261, Training Accuracy= 0.453
Step 1000, Minibatch Loss= 1.7176, Training Accuracy= 0.414
Step 1200, Minibatch Loss= 1.6452, Training Accuracy= 0.477
Step 1400, Minibatch Loss= 1.5781, Training Accuracy= 0.570
Step 1600, Minibatch Loss= 1.3760, Training Accuracy= 0.570
Step 1800, Minibatch Loss= 1.4557, Training Accuracy= 0.523
Step 2000, Minibatch Loss= 1.3412, Training Accuracy= 0.594
Step 2200, Minibatch Loss= 1.3234, Training Accuracy= 0.562
Step 2400, Minibatch Loss= 1.2639, Training Accuracy= 0.641
Step 2600, Minibatch Loss= 1.2196, Training Accuracy= 0.625
Step 2800, Minibatch Loss= 1.2287, Training Accuracy= 0.609
Step 3000, Minibatch Loss= 1.1813, Training Accuracy= 0.633
Step 3200, Minibatch Loss= 1.2933, Training Acc